In [2]:
%run data.ipynb

In [6]:
class Food():
    def __init__(self, name):
        data = foods_df.loc[name]
        self.name = name
        self.tier = data['Tier']
        
        target, n_targets, effect, effect_info = foods_df.loc[name, 'Target':'Effect_info']
        self.target = target.__name__
        self.effect = effect.__name__
        self.target_func = target(n_targets=n_targets)
        self.effect_func = effect(None, None, effect_info, 1)
        
        self.cost = 3
        if self.name == 'Sleeping Pill':
            self.cost = 1
        self.targeted = isinstance(target, SelectedPet)
        self.is_frozen = False
        
    def __repr__(self):
        return self.name + (' (frozen)' if self.is_frozen else '')
    
    def use(self, targets, shop, cat_mult=1, verbose=False):
        if self.target != 'SelectedPet':
            targets = self.target_func(shop, True, [])
        
        self.effect_func(targets=targets, gamestate=shop, in_team1=True)
        # cat aura
        if self.effect == 'ModifyStats' and cat_mult > 1:
            for _ in range(cat_mult - 1):
                self.effect_func(targets=targets, gamestate=shop, in_team1=True)
        
        for pet in targets:
            pet.trigger_dict['EatsShopFood'] += 1
            
        if verbose: print(f'Bought {self.name} for {[str(target) for target in targets]}')